In [24]:
from problems.HS100 import HS100, CallableClass
import methods.methods as methods
from typing import Tuple, Union, List, Type
from copy import deepcopy
import pandas as pd
import numpy as np
from numpy import number
from scipy.stats.qmc import LatinHypercube as lhs
from scipy.stats.qmc import scale
from smt.surrogate_models import KRG

In [25]:
def simple_experiment(problem: dict, n_samples: int) -> pd.DataFrame:
    """ Simple wrapper to create an experiment
    
    Parameters
    ----------
    problem : dict
        The variables and constraints given

    n_sample: int
        The number of sites to generare
   
    Returns
    -------
    pd.DataFrame
        The DataFrame of evaluated experimental sites with constraint violation
    """

    variables = list(problem["variables"].keys())
    nind = len(variables)
    # Get all the bounds for the variables
    bounds = np.array([problem["variables"][var]["bounds"] for var in variables])
    # Generate the experiment 
    lhs_instance = lhs(
    nind,
    scramble=True,
    strength=1,
    optimization=None,
    seed=1232,
    )
    # Get the experiment
    normalized_array = lhs_instance.random(n_samples)
    # Scale using the bounds
    exp_array = scale(normalized_array, bounds[:, 0], bounds[:, 1])
    # Create the DataFrame
    exp_df = pd.DataFrame(data=exp_array, columns=variables)
    return(exp_df)


In [26]:
def testing(local_eval: CallableClass, num_sites: int, verbose=True) -> pd.DataFrame:
    """ Simple wrapper to create an experiment, evaluate the passed in function

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample: int
            The number of sites to generare
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    eps = 1.e-6
    local_problem = local_eval.problem()
    exp_data = simple_experiment(local_problem, num_sites)
    local_eval(exp_data)
    my_constraint_calculator = methods.ConstraintCalculator(local_problem)
    exp_data['__conviol__'] = my_constraint_calculator(exp_data)
    exp_data['__State__'] = pd.cut(exp_data['__conviol__'], [-np.inf, eps, 100*eps, np.inf], include_lowest=True, labels=['Feasible', 'Nearly Feasible', "Infeasible"]).astype("str")
    feasible_sites = (exp_data['__State__'] == 'Feasible').sum()
    percentage = feasible_sites/num_sites * 100
    if verbose: 
        print(f"Test evaluator {local_eval.name} with {len(local_problem['variables'])} variables {percentage}% feasible sites")
    return (exp_data)

In [27]:
def advanced_testing(local_eval: CallableClass, num_sites_training: int, num_sites_testing : int, verbose=True) -> pd.DataFrame:
    """ Wrapper to create an experiment, evaluate the passed in function, create a surrogate model, evaluate model

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample_training: int
            The number of sites to train the sm on

        n_sample_testing: int
            The number of sites to test the sm
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    # get training data
    exp_data = testing(local_eval, num_sites_training, False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = exp_data['__conviol__'].to_numpy()

    # create model
    sm = KRG(theta0=[1e-2])
    sm.set_training_values(xt, yt)
    sm.train()

    # get testing data
    eps = 1e-6
    x = simple_experiment(local_eval.problem(), num_sites_testing).to_numpy()
    y = data=sm.predict_values(x).flatten()
    feasible_points = pd.DataFrame(data=x[y < eps], columns=variables)
    
    # evaluate model on testing data
    local_eval(feasible_points)
    my_constraint_calculator = methods.ConstraintCalculator(local_eval.problem())
    feasible_points['__conviol__'] = my_constraint_calculator(feasible_points)
    feasible_points['__State__'] = pd.cut(feasible_points['__conviol__'], [-np.inf, eps, 100*eps, np.inf], include_lowest=True, labels=['Feasible', 'Nearly Feasible', "Infeasible"]).astype("str")
    feasible_sites = (feasible_points['__State__'] == 'Feasible').sum()
    percentage = feasible_sites/feasible_points.shape[0] * 100
    if verbose: 
        print(f"Test evaluator {local_eval.name} with {len(local_eval.problem()['variables'])} variables {percentage}% feasible sites")
    return (feasible_points) 
    

In [28]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = int((nind+1)*(nind+2)/2)
test_data = advanced_testing(hs100, num_sites, num_sites**2)
test_data

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 36
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  1.7225320
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1296
   
   Predicting ...
   Predicting - done. Time (sec):  0.0342100
   
   Prediction time/pt. (sec) :  0.0000264
   
Test evaluator hs100 with 7 variables 6.329113924050633% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,-7.181532,-3.311914,0.301812,7.204311,-8.671379,9.625825,7.805377,4.256783e+06,-501.644219,325.419872,-143.289546,-108.364696,1.865242,Infeasible
1,3.514442,-2.624388,0.104009,-0.510216,-6.025205,-2.117039,7.531574,4.832267e+05,-11.031146,259.648900,141.641893,9.448470,0.011031,Infeasible
2,3.318404,-2.549490,-2.904715,-3.663962,6.221996,3.465331,4.738813,5.824701e+05,-103.673330,191.931910,79.036201,-58.002275,0.589215,Infeasible
3,-2.177121,4.135016,-0.788500,-9.485709,-0.119023,0.884121,6.230232,3.148373e+03,-1118.071853,287.984165,274.127261,-0.196563,1.118074,Infeasible
4,-9.922751,-4.707717,-2.986161,8.503304,3.427768,-2.728121,9.280469,2.563583e+04,-1846.841246,271.335274,431.648559,-177.974614,2.564823,Infeasible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,-0.537764,-2.662500,-4.665996,5.243581,-0.265627,6.099897,9.850622,1.105545e+04,-128.322594,70.527490,56.832147,30.364050,0.128323,Infeasible
75,-8.896044,-1.781405,-4.126569,5.353170,3.695542,2.553548,8.006554,3.114840e+04,-190.467559,177.673170,422.364392,-230.942233,2.317263,Infeasible
76,-0.007259,0.809088,-3.051825,1.916378,-5.920891,-4.415261,8.329398,4.369814e+05,143.680555,178.649944,145.180325,94.399958,0.000000,Feasible
77,3.155760,-4.156159,0.267015,9.451520,-6.833737,3.761715,9.040193,1.026365e+06,-1111.478643,255.379929,93.562404,-15.965510,1.122887,Infeasible
